# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import os

# 设置环境变量，下载器就会使用镜像站
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
pip install -r requirements.txt --trusted-host mirrors.aliyun.com -i http://mirrors.aliyun.com/pypi/simple/

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [4]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.545 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [5]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [6]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [7]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [8]:
print(response[0].split("### Response:")[1])


<think>
嗯，我现在要处理这位28岁的程序员的病例。他最近工作到半夜，感觉头晕、脖子疼，有时还会恶心。首先，我需要考虑他的工作情况，因为他每天工作到很晚，这可能对他的身体有影响，比如疲劳、睡眠不足，这些都是可能导致头晕和恶心的原因。

然后，我想到可能他有颈椎问题，尤其是如果他长时间低头看电脑，可能会压迫颈椎，导致疼痛和不适。这种情况比较常见，尤其是年轻人长时间使用电子设备的人群。

接下来，我需要考虑他的症状是否符合某种特定的疾病。头晕和恶心可能与前庭系统有关，比如梅尼埃病，这是一种常见的神经系统疾病，可能导致恶心、呕吐、头晕等症状。但梅尼埃病通常会伴随耳部症状，比如听力减退或耳痛，如果他没有这些症状，可能排除这个可能性。

另外，我想到他可能有睡眠障碍，比如失眠或者睡眠质量不好，导致白天精力不足，出现头晕和恶心。睡眠不足会导致大脑供血不足，从而产生疲劳感和不适感。

我还应该考虑其他可能的健康问题，比如压力过大、焦虑或者抑郁，这些心理因素也可能导致类似的症状。但通常情况下，这些症状可能更持久，而不仅仅是工作到深夜引起的。

然后，我需要建议他做一些检查，比如血常规、头部影像学检查（如CT或MRI）、神经功能检查，以及评估他的睡眠质量。这些检查可以帮助确定具体的原因。

此外，我应该建议他进行一些缓解症状的活动，比如放松训练、深呼吸或者改变工作姿势，如果这些方法无效，可能需要进一步的医疗建议。

总的来说，我需要综合考虑他的症状、生活方式以及可能的健康问题，然后给出一个全面的评估和建议。
</think>

根据您的描述，这位28岁的程序员出现头晕、脖子疼和恶心，可能与多种因素相关。以下是一份分步的分析报告：

1. **初步评估**：
   - **症状类型**：头晕、脖子疼痛、恶心。
   - **可能原因**：
     - 工作压力和疲劳。
     - 颈部姿势不舒适。
     - 心血管或神经系统问题。
     - 梅尼埃病（需考虑是否有耳部症状）。
     - 睡眠障碍。

2. **建议检查**：
   - **血液检查**：评估贫血或低血糖。
   - **头部影像学检查**（如CT或MRI）：评估颈椎或头部结构。
   - **神经功能评估**：评估神经功能完整性。
   - **睡眠质量评估**：评估是否存在失眠或睡眠障碍。



---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [9]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [10]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [11]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [12]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [13]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-3): 4 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [14]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 80,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [15]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 80
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 80
O^O/ \_/ \    Batch size per device = 57 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (57 x 2 x 1) = 114
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 80
O^O/ \_/ \    Batch size per device = 51 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.116200
2,3.011000
3,3.052500
4,2.982800
5,2.958200
6,2.961600
7,2.872700
8,2.705600
9,2.494400
10,2.487400


TrainOutput(global_step=80, training_loss=2.112584951519966, metrics={'train_runtime': 2059.2829, 'train_samples_per_second': 3.108, 'train_steps_per_second': 0.039, 'total_flos': 2.6008130525110272e+17, 'train_loss': 2.112584951519966})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [16]:
model.save_pretrained("qwen-7b_lora_model")

In [17]:
tokenizer.save_pretrained("qwen-7b_lora_model")

('qwen-7b_lora_model/tokenizer_config.json',
 'qwen-7b_lora_model/special_tokens_map.json',
 'qwen-7b_lora_model/chat_template.jinja',
 'qwen-7b_lora_model/tokenizer.json')

In [18]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [20]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人已经有5天的急性阑尾炎了，腹痛虽然稍微减轻了，但还是有点疼痛。而且发热也是个问题。现在体检的时候，右下腹有一个压痛的包块。嗯，这说明可能有感染了。

首先，我得想到阑尾炎通常会伴随感染，所以这个包块可能就是感染的迹象。不过，包块可能不是阑尾炎的唯一表现，也可能是其他问题，比如急性胆囊炎或者急性胃炎。

哦，急性阑尾炎常伴有右下腹的压痛，这可能是包块的信号。但是，如果包块有压痛，而且发热的话，那情况就不一样了。发热一般和感染有关，比如细菌感染，这让我觉得可能不是阑尾炎。

再想想，阑尾炎一般不会有发热，除非是细菌感染，但这种情况下，包块和发热都会出现。那这个包块是不是代表感染了某种细菌呢？

查一下，如果包块是感染性的，那我们需要考虑可能的感染原因，比如胆囊炎或者急性胃炎，这些都会导致发热和包块。

不过，别急着下结论，得再仔细想想。急性阑尾炎一般不会伴随发热，除非是细菌感染。那这个包块和发热可能指向细菌感染，这让我怀疑是急性胆囊炎。

嗯，我需要进一步确认，但根据目前的信息，这个包块和发热提示可能是细菌感染引起的急性胆囊炎。

不过，也有可能是其他原因，比如急性胃炎，虽然通常不伴随发热，但有时候也会有，所以也不能排除。

啊，这个包块的压痛和发热，让我更倾向于细菌感染的可能性，尤其是急性胆囊炎。

好吧，看来我得按照这个思路来处理，可能需要进一步检查，比如B超或者做一个影像学检查，看看包块的具体情况。

总之，这个包块和发热提示可能是细菌感染引起的急性胆囊炎，我得记住这个情况，以便采取相应的治疗措施。
</think>
根据您的描述，这个病人有5天的急性阑尾炎病史，腹痛虽然稍有减轻，但仍然存在发热和右下腹的压痛包块。这些症状提示可能存在细菌感染，尤其是急性胆囊炎的可能性较大。急性阑尾炎通常不会伴随发热，除非是细菌感染，而发热和包块的压痛更常与急性胆囊炎相关联。因此，建议进行进一步的影像学检查，如B超，以明确包块的性质和位置。如果包块呈感染性表现，可能需要考虑急性胆囊炎的诊断。建议根据影像学结果和症状变化采取相应的治疗措施。



In [21]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [22]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
60岁男性患者，右胸疼，X线显示右肋膈角消失，提示可能有胸腔积液。这种情况下，肺结核伴右侧胸腔积液，我得考虑一下胸水的性质。胸水的性质可能影响诊断和治疗，比如是否是脓液还是清液，或者有其他成分。通常，了解胸水的成分对于治疗和诊断非常关键。

首先，我想到常规的实验室检查，比如血常规和肝肾功能检查。这些检查可以帮助我们了解患者的总体健康状况，以及是否存在感染、炎症或者其他代谢问题。但是，它们可能无法直接提供关于胸水的详细信息。

嗯，如果我想更深入地了解胸水的性质，可能需要更直接的分析。实验室检查中，血培养和胸水培养是常用的手段。血培养可以检测是否存在特定病原体，比如肺炎链球菌，而胸水培养则可以分析胸水中的病原体，如细菌、病毒或者真菌。这些信息对于判断胸水的性质和进行针对性治疗非常重要。

再想想，还有没有其他更直接的方法呢？比如影像学检查，比如超声，可以提供胸水的外观信息。但影像学检查只能大致了解胸水的液量和外观，不能提供具体的成分分析。

哦，对了，还有一个办法，那就是胸水穿刺，获取胸水样本进行进一步的分析。通过胸水穿刺，可以将胸水样本送至实验室进行各种分析，如培养、基因检测、病毒检测等。这些检测可以提供关于胸水的成分和病原体的信息，这对诊断和治疗非常有帮助。

不过，胸水穿刺可能在临床上不太常见，因为操作相对复杂，成本也较高。不过，对于了解胸水的性质，这是最直接有效的方法之一。

综上所述，为了更准确了解胸水的成分和性质，血培养和胸水培养是最直接的实验室检查手段。此外，胸水穿刺也是有效的方法，可以提供更详细的信息。
</think>
对于一名66岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，了解胸水的性质需要进行更深入的实验室检查。以下是可能对了解胸水性质有帮助的实验室检查：

1. **血培养**：通过血培养可以检测是否存在特定病原体，如肺炎链球菌、结核菌等，这有助于了解胸水中的病原体种类。

2. **胸水培养**：胸水培养是直接分析胸水成分和病原体的常用方法。通过培养可以确定胸水中的病原体类型，如细菌、病毒或真菌，从而明确胸水的性质。

3. **胸水穿刺**：胸水穿刺可以获取胸水样本，进一步进行基因检测、病

In [23]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位28岁的程序员，长期熬夜，最近突然觉得头晕目眩，有点恶心，这让我想到他可能有高血压。不过，也有可能是其他什么问题呢？

首先，我得考虑他最近的生活习惯。长期熬夜，可能影响了他的血压。长时间处于黑暗和低光环境下，这会降低身体对光的敏感度，从而降低视网膜中的视杆状细胞的兴奋性，这可能引起视杆状细胞减少，进而导致血压升高。

但是，高血压也会伴随一些症状，比如口渴、多尿、多饮和视物模糊。这位患者没有提到这些症状，所以可能不是高血压。

那，他最近有没有其他症状呢？比如头痛、口渴或视物模糊？如果他没有这些症状，那可能不是高血压。

再想想，他长期熬夜，可能还有其他原因。比如，他可能在睡眠质量上出了问题。睡眠不足，会导致身体的血压升高。

也有可能是他的工作压力大，导致压力性高血压。压力确实会导致血压升高，尤其是长时间处于紧张状态。

不过，患者没有提到压力性高血压的症状，比如口渴、多尿、视物模糊，所以可能不是压力性高血压。

还有，他最近突然感觉头晕目眩，这可能是因为血压上升引起的脑供血不足。但是，如果没有口渴、多尿和视物模糊等症状，可能不是高血压。

再想想，他最近可能有其他疾病。比如，他可能有糖尿病，或者他可能有睡眠障碍，比如失眠或者睡眠呼吸暂停。

也有可能是其他原因，比如他最近开始吃某些药物，或者他有某些生活习惯导致血压异常。

嗯，看来我得考虑更多的可能性，比如睡眠障碍、压力性高血压或者其他慢性病。

不过，从目前的症状来看，他可能有高血压，但也需要排除其他可能性。

或者，他的头晕目眩可能是由于其他原因引起的，比如脑供血不足、贫血、脑水肿等。

不过，患者没有提到这些症状，所以可能还是高血压的可能性更大。

嗯，综合来看，虽然他没有提到一些高血压的症状，但长期熬夜可能影响血压，导致头晕目眩。

所以，我还是倾向于认为他可能有高血压，虽然需要进一步的检查来确认。
</think>
根据这位28岁程序员的描述，他长期熬夜且最近突然出现头晕目眩，这种情况可能与高血压有关。长期熬夜可能导致视杆状细胞减少，进而引发血压升高。然而，患者没有提到口渴、多尿、视物模糊等高血压的典型症状，这可能需要进一步确认。建议进行详细的体检和血压监测，以明确诊断。如果血压异常